In [ ]:
import yaml
from pathlib import Path
import pandas as pd
import numpy as np
from aicsimageio import AICSImage
import sys
sys.path.append("/pasteur/u/jmhb/confocal-2d-video-processing")
import segmentation_core as sc
import seg_eval as se
import pipeline_utils as pu
import utils
import importlib

import matplotlib.pyplot as plt
import matplotlib.animation as animation

# project path 
c = yaml.safe_load(open("/pasteur/u/jmhb/confocal-2d-video-processing/config.yaml", "r"))
PATH_PROJECT = Path(c['path_project'])
# data path
c = yaml.safe_load(open("/pasteur/u/jmhb/confocal-2d-video-processing/config.yaml", "r"))
c['path_data'] = "/pasteur/data/hiPSCs_January2022" # hack
PATH_DATA = Path(c['path_data'])
# PATH_FNAMES = PATH_DATA / "fname-lookup.csv"
PATH_FNAMES = PATH_DATA / "fname-lookup-timelapse.csv"
# PATH_FNAMES = PATH_DATA / "fname-lookup-zstack.csv"

PATH_SEG_FUNCS = PATH_PROJECT / "seg-func-map.yaml"
PATH_EVALS = Path("/pasteur/u/jmhb/confocal-2d-video-processing/analyses/jan22-neural-diff/eval-pdfs")

## choose the images to segment 
channel="nuclei"
frame=50


In [ ]:
### Filter on the paths
if 1:
    df_fnames = pd.read_csv(PATH_FNAMES).set_index('index')
    df_fnames_filt = df_fnames.query("(g1=='D0') | (g1 in ('AP24','AP48') & g2 in ('CTRL','CTRLOAC','CTRLAC')) ")    
else:
    # else only remove WT since it's bad
    df_fnames_filt = df_fnames.query("(g1!='WT')")    

df_fnames_filt['path_file'] = str(PATH_DATA) +"/"+ df_fnames_filt['folder']+"/"+df_fnames_filt['fname']
print(df_fnames_filt.shape)

#### load img data

In [ ]:
# get the middle frame 
imgs = [] 
channel='nuclei'
for idx, row in df_fnames_filt.iterrows():
    print(idx, end=" ")
    fname = row.path_file
    img_obj = AICSImage(fname)
    ch_idx_lookup = pu.get_channel_idx_lookup(img_obj.channel_names)
    ch_idx = ch_idx_lookup[channel]
    x = img_obj.get_image_data("TCZYX")
    frame = x.shape[0]//2
    img=x[frame, ch_idx,[0]].copy()
    print(f"Shape {x.shape}, getting frame {frame}, ch_idx {ch_idx}")

    del x
    imgs.append(img)

lookup_idx_to_imgsidx = dict(zip(df_fnames_filt.index, np.arange(len(df_fnames_filt))))    
imgs_D0 = np.concatenate(imgs[:32]).astype(np.float32)
imgs_AP = np.concatenate(imgs[32:]).astype(np.float32)

In [ ]:
from torchvision.utils import make_grid 
nimgs=10
figsize=(40,40)
grid = make_grid(torch.Tensor(imgs_D0).unsqueeze(1), nimgs)[0]
f,axs = plt.subplots(1,1,figsize=figsize)
axs.imshow(grid, cmap='gray')
plt.show()
grid = make_grid(torch.Tensor(imgs_AP).unsqueeze(1), nimgs)[0]
f,axs = plt.subplots(1,1,figsize=figsize)
axs.imshow(grid, cmap='gray')

#### segmentation info

In [ ]:
# load seg info 
## load segmentation functions
seg_funcs = sc.load_segmentation_functions(fname=PATH_SEG_FUNCS)

## segmentation config file 
config_fname =  "./configs/example-config.yaml"
seg_config = sc.load_segmentation_config_from_file(fname=config_fname, seg_funcs=seg_funcs, verbose=0)


In [ ]:
# from aicssegmentation.core import utils 
# utils.hole_filling?

In [ ]:
import yaml
dct = yaml.safe_load("""
nuclei:
    seg: 
        pipeline: 
            AC: 
                scaling_param: [0,15]
            G2:
                sigma: 3.5
            MO: 
                object_minArea: 1000
                global_thresh_method: ave_tri_med # tri, med, ave_tri_med
            HF: 
                hole_min: 0
                hole_max: 1200
            #WD: {}
            S:
                min_size: 200
                connectivity: 1
        ops: 
            inplace: False
""")
seg_config[channel] = dct[channel]


In [ ]:
importlib.reload(sc)
all_figs = []
nlim = 100
df_test = df_fnames_filt.iloc[::-1]
for i, (idx, row) in enumerate(df_test.iterrows()):
    if i>nlim:break
    print(idx)
    fname =row.path_file
    ### record info about this file 
    print(fname)

    f = plt.figure(figsize=(10,1)); plt.axis('off')
    # plt.text(0.5,0.5,msg,ha='center',va='center', fontsize='xx-large', fontstretch='expanded')
    all_figs.append(f)
    plt.close()
    
    ### get the image and make a copy
    img_in = imgs[lookup_idx_to_imgsidx[idx]]
    img = img_in.copy()

    ### segmentation params 
    show_whole_seg, show_sliding_window_masks, show_sliding_window = True, False, True
    # Warning - this is only for display. 
    # The actual AC norm used in segmentation is the one listed in the YAML config file
    ac_norm_params=[[0,15]]   
    gs_smooth_params = [1,2]
    es_smooth_params = [
        dict(niter=5,kappa=1.2,gamma=0.1),
    #     dict(niter=40,kappa=1.2,gamma=0.1)
                       ]
    # segmentation 
    seg, imgs_inter = sc.seg_2d(img[0], seg_config, seg_funcs, ch_name=channel, inplace=False, save_inters=True)

    # show whole image
    if show_whole_seg:
        f, axs = plt.subplots(1,2,figsize=(20,20))
        axs[0].imshow(img[0], cmap='gray')
        axs[1].imshow(seg[0], cmap='gray')
        plt.close()
        all_figs.append(f)
        display(f)

    # sliding window params
    window_sz, stride = 800, 800
    min_pixels, figsize_mult, max_display_cols, do_plot_dividing_lines = 10, 7, 4, 5
    f, show_mask, noshow_mask = pu.compare_seg_windows(img, seg, imgs_inter=imgs_inter,
                                            max_display_cols=max_display_cols, figsize_mult=figsize_mult,do_plot_dividing_lines=do_plot_dividing_lines,
                                            ac_norm_params=ac_norm_params,gs_smooth_params=gs_smooth_params, window_sz=window_sz, stride=stride,
                                                      es_smooth_params=es_smooth_params)
    all_figs.append(f)
    display(f)
    assert np.array_equal(img, img_in), "Warning: the image got modified inplace somewhere"

    if show_sliding_window_masks:
        f_tmp, axs = plt.subplots(1,2, figsize=(20,20))
        axs[0].imshow(noshow_mask, cmap='gray');    axs[0].set(title="Ignored windows")
        axs[1].imshow(show_mask, cmap='gray')  ;    axs[1].set(title="Included widnows")
        display(f_tmp)
    
    
    

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
fname_eval = PATH_EVALS / f"{channel}-4samples.pdf"
with PdfPages(fname_eval) as pdf:
    for f in all_figs:
        pdf.savefig(f)
    

In [ ]:
from pathlib import Path
PATH_DATA = Path("/pasteur/data/hiPSCs_January2022/")
PATH_FNAMES = PATH_DATA / "fname-lookup-timelapse.csv"
import pandas as pd
df_fnames = pd.read_csv(PATH_FNAMES).set_index('index')
df_fnames['path_file'] = str(PATH_DATA) +"/"+ df_fnames['folder']+"/"+df_fnames['fname']
# df_fnames=df_fnames.loc[["AP" in s for s in df_fnames.index]]


In [ ]:
df_fnames.query("g1 in ('D0','D7','D14','D21',)")


In [ ]:
df_fnames.loc[["AP" in s for s in df_fnames.index]].g2.unique()
for i, (idx, row) in enumerate(df_fnames.query("g2 in ('CTRL','CTRLAC','CTRLOAC')").iterrows()):
    print(i, row.path_file)

In [ ]:
for i, row in df_fnames.iterrows():
    print(i, row.path_file)

In [ ]:
row

In [ ]:
from aicsimageio import AICSImage

f="/pasteur/data/hiPSCs_January2022/Trial_ApoE-Knock-down-in-hiPSCs-undifferentiated/ApoE-Knock-Down-48hrs/48hrs-Ctrl_7_Unmixing.czi"
aics_img = AICSImage(f)



In [ ]:
import os
f_img = "/pasteur/data/hiPSCs_January2022/Differentiation-into-cortical-neuron_January2022/hiPSCs_KOLF2.1J_PB-TO-hNGN2clone1-TIME_LAPSE/hiPSCs-undifferentiated_day0/KOLF2.1J-PBTOhNGN2clone1-2_02_Unmixed.czi"
f_seg = "/pasteur/data/hiPSCs_January2022/seg-framewise/feb23/Differentiation-into-cortical-neuron_January2022/hiPSCs_KOLF2.1J_PB-TO-hNGN2clone1-TIME_LAPSE/hiPSCs-undifferentiated_day0/KOLF2.1J-PBTOhNGN2clone1-2_02_Unmixed.czi"

f_img = "/pasteur/data/hiPSCs_January2022/Trial_ApoE-Knock-down-in-hiPSCs-undifferentiated/ApoE-Knock-Down-48hrs/48hrs-Ctrl_+_oleic_acid_09_Unmixing.czi"
f_seg = "/pasteur/data/hiPSCs_January2022/seg-framewise/feb23/Trial_ApoE-Knock-down-in-hiPSCs-undifferentiated/ApoE-Knock-Down-48hrs/48hrs-Ctrl_+_oleic_acid_09_Unmixing.czi"

aics_img = AICSImage(f_img)
aics_seg = AICSImage(f_seg)
img = aics_img.data
seg = aics_seg.data


In [ ]:
aics_img.dims

In [ ]:
import matplotlib.pyplot as plt
ch_names = aics_img.channel_names
for i in range(8):
    print(ch_names[i])
    f,axs = plt.subplots(1,2,figsize=(10,10))

    axs[0].imshow(img[0,i,10],cmap='gray')
    axs[1].imshow(seg[0,i,10],cmap='gray')
    plt.show()